# Federated PCA  with Flex  for Anomaly Detection

In this notebook we show how to use an PCA model for anomaly detection with federated learning using the flexible

First we do all the imports needed.

In [1]:
from flexanomalies.utils import PCA_Anomaly
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.aggregators_pca import aggregate_pca
from flexanomalies.pool.primitives_pca import (
    build_server_model_pca,
    copy_model_to_clients_pca,
    train_pca,
    set_aggregated_weights_pca,
    get_clients_weights_pca,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool

 Load data, define model parameters and define model

In [2]:
model_params = {"n_components": 4, "contamination": 0.1}

In [4]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

In [5]:
model = PCA_Anomaly(**model_params)

Once the data  is loaded, we have to federate it. For this we use the FLEX library. There are two ways to federate the data, using an IID distribution or a non IID distribution. For the IID distribution we can use the ìid_distribution function of 

FedDataDistribution. If we use a non-IID distribution, it is necessary to use a custom configuration, such as the one used in the federate_data function. For more information, go to the FLEX library workbooks, and take a look at the Federate Data with 

FLEXible notebooks.

## Creating the federated architecture

When creating the federated architecture, we use FlexPool. Since we are running a client-server architecture, we use the client_server_architecture function. 

In [6]:
flex_dataset = federate_data(5, X_train, y_train)

In [7]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="pca_server",
    init_func=build_server_model_pca,
    model=model,
)

## Run the federated learning experiment and Evaluate

Now, we can run the federated experiment for multiple rounds using the decorators. 

Once the model is trained, we need to evaluate it at the server level.


In [ ]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_pca, pool.clients)
    pool.clients.map(train_pca)
    pool.aggregators.map(get_clients_weights_pca, pool.clients)
    pool.aggregators.map(aggregate_pca)
    pool.aggregators.map(set_aggregated_weights_pca, pool.servers)
output_model = pool.servers._models["pca_server"]["model"]
output_model.evaluate(X_test, y_test)

In [ ]:
output_model.result_metrics_

In [10]:
save_experiments_results(
    "PCA",
    output_model,
    "test_PCA_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)

## End of Notebook